This notebook can be run interactively or submitted to SLURM cluster at DLS using the notebook entitled **SLURM_notebook_submit.ipynb** 

We do the following:
- Load the K3-EELS nultiframe data
- Save individual frames LL and HL data
- Save ZLP positions in the FOV
- Align all frames with respect to the ZLP and sum to get one aligned data


Creator(s):
- Mohsen Danaie (ePSIC)- mohsen.danaie@diamond.ac.uk 

TODO: Add drift correction

In [ ]:
# %matplotlib widget  

## Comment out the above magic command for clustrer submission

import py4DSTEM
import hyperspy.api as hs
import numpy as np
import h5py
print(np.__version__)
print(h5py.__version__)
import exspy
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('/dls_sw/e02/software/epsic_tools')
import epsic_tools.api as ep

file_path_HL=/dls/e01/data/2025/mg38764-5/raw/SCD2070/InSitu (1)/STEM SI_EELS HL SI.dm5

file_path_LL=/dls/e01/data/2025/mg38764-5/raw/SCD2070/InSitu (1)/STEM SI_EELS LL SI.dm5

images_path=/dls/e01/data/2025/mg38764-5/raw/SCD2070/InSitu (1)/STEM SI_HAADF Image.dm4

cal_ll_path=/dls/e01/data/2025/mg38764-5/raw/SCD2070/InSitu (4)/STEM SI_EELS LL SI.dm4

cal_hl_path=/dls/e01/data/2025/mg38764-5/raw/SCD2070/InSitu (4)/STEM SI_EELS HL SI.dm4

save_path=/dls/e01/data/2025/mg38764-5/processing/SI(1)


In [ ]:
# Load dm4 files for axes info

d_cal_hl = hs.load(cal_hl_path)
d_cal_ll = hs.load(cal_ll_path)

In [ ]:
print(d_cal_hl.axes_manager)

# Loading data

In [ ]:
# file_path = r'/dls/e01/data/2025/mg38764-5/raw/SCD2070/InSitu (1)/STEM SI_EELS HL SI.dm5'
# file_path_LL = r'/dls/e01/data/2025/mg38764-5/raw/SCD2070/InSitu (1)/STEM SI_EELS LL SI.dm5'
# images_path = '/dls/e01/data/2025/mg38764-5/raw/SCD2070/InSitu (1)/STEM SI_HAADF Image.dm4'

In [ ]:
images_path

In [ ]:
im = hs.load(images_path)

In [ ]:
f = h5py.File(file_path_HL, "r")

In [ ]:
def print_attrs(name, obj):
    print(f"Name: {name}")
    for key, value in obj.attrs.items():
        print(f"   Attributes: {key} => {value}")
f.visititems(print_attrs)

In [ ]:
MyInSituData = ep.load_k3_data.InSitu_K3_Reader(file_path_HL) 
MyInSituData_LL = ep.load_k3_data.InSitu_K3_Reader(file_path_LL) 

In [ ]:
frames_num = MyInSituData.GetTotalFrameNum()

In [ ]:
# def print_attrs(name, obj):
#     print(name)
#     for key, val in obj.attrs.items():
#         print('%s %s' % (key, val))
        
# f = h5py.File(file_path,'r')
# f.visititems(print_attrs)

# load all frames and sum 

In [ ]:
ll_list = []
hl_list = []
for i in range(0,frames_num):
    print(i)
    _d_hl = MyInSituData.GetNthFrame_lazy(i)
    _d_ll = MyInSituData_LL.GetNthFrame_lazy(i)
    _d_hl = exspy.signals.LazyEELSSpectrum(_d_hl)
    _d_ll = exspy.signals.LazyEELSSpectrum(_d_ll)
    _d_hl = _d_hl.transpose(signal_axes=[1], navigation_axes=[2,0])
    _d_ll = _d_ll.transpose(signal_axes=[1], navigation_axes=[2,0])
    _d_hl.axes_manager = d_cal_hl.axes_manager
    _d_ll.axes_manager = d_cal_ll.axes_manager
    _d_ll.compute()
    _d_hl.compute()
    _d_ll.save(os.path.join(save_path, f'raw_ll_frame_{i}.hspy'))
    _d_hl.save(os.path.join(save_path, f'raw_hl_frame_{i}.hspy'))
    ll_list.append(_d_ll)
    hl_list.append(_d_hl)


In [ ]:
d_cal_ll.sum().plot()

In [ ]:
raw_data_ll_sum = sum(signal.data for signal in ll_list)
raw_data_ll_sum = exspy.signals.EELSSpectrum(raw_data_ll_sum)
raw_data_ll_sum.save(os.path.join(save_path, 'sum_ll_raw.hspy'), overwrite=True)
raw_data_ll_sum.axes_manager = d_cal_ll.axes_manager
# raw_data_ll_sum.compute()
raw_ll_sum = raw_data_ll_sum.sum()
raw_ll_sum.isig[-5.0:5.0].plot()
fig = plt.gcf()
fig.savefig(os.path.join(save_path, 'sum_ll_raw.png'))

In [ ]:
# Estimate ZLP pos
zlp_pos = []
for i in range(0,frames_num):
    print(i)
    _temp_ll = ll_list[i]
    # _temp_ll.compute()
    _zlp = _temp_ll.estimate_zero_loss_peak_centre()
    _zlp.save(os.path.join(save_path, f'zlp_pos_frame_{i}.hspy'), overwrite=True)
    _zlp.plot()
    fig = plt.gcf()
    fig.savefig(os.path.join(save_path, f'zlp_pos_frame_{i}.png'))
    zlp_pos.append(_zlp)


In [ ]:
# Aligning in energy:
aligned_ll = []
aligned_hl = []
for i in range(0,len(ll_list)):
    print(i)
    _temp_ll = ll_list[i]
    _temp_hl = hl_list[i]
    _temp_ll.align_zero_loss_peak(calibrate=False,
                                also_align=[_temp_hl], 
                                subpixel=False,
                                crop=False,
                                signal_range=(-5.0,5.0))

    aligned_ll.append(_temp_ll)
    aligned_hl.append(_temp_hl)

In [ ]:
aligned_stack = hs.stack(d.sum() for d in aligned_ll)

In [ ]:
aligned_stack.isig[-10.:7.0].plot()

In [ ]:
fig = plt.gcf()
fig.savefig(os.path.join(save_path, f'stacked_aligned_ll_data.png'))

In [ ]:
margin = 100 # offset from the ZLP to crop
energy_ax_size = 3000 # number of channels in the final aligned data to retain

zlp_indices = []
for i, data in enumerate(aligned_ll):
    if i==0:
        zlp_ind = np.argmax(data.sum().data)
        zlp_indices.append(zlp_ind)

        aligned_ll_sum = data.isig[int(zlp_ind - margin):int(zlp_ind - margin + energy_ax_size)]
        aligned_hl_sum = aligned_hl[i].isig[int(zlp_ind - margin):int(zlp_ind - margin + energy_ax_size)]
        # print(aligned_ll_sum.axes_manager)
    else:
        print(i)
        zlp_ind = np.argmax(data.sum().data)
        zlp_indices.append(zlp_ind)
        # print(zlp_ind)
        aligned_ll_sum += data.isig[int(zlp_ind - margin):int(zlp_ind - margin + energy_ax_size)]
        aligned_hl_sum += aligned_hl[i].isig[int(zlp_ind - margin):int(zlp_ind - margin + energy_ax_size)]



In [ ]:
# Doing this one final time to push ZLP pos to zero eV
aligned_ll_sum.align_zero_loss_peak(calibrate=True,
                                also_align=[aligned_hl_sum], 
                                subpixel=False,
                                crop=False,
                                signal_range=(-5.0,5.0))

In [ ]:
aligned_ll_sum.sum().plot()

In [ ]:
fig = plt.gcf()
fig.savefig(os.path.join(save_path, f'aligned_ll_sum_data_final.png'))

In [ ]:
aligned_ll_sum.save(os.path.join(save_path, 'aligned_ll_data.hspy'), overwrite=True)
aligned_hl_sum.save(os.path.join(save_path, 'aligned_hl_data.hspy'), overwrite=True)